In [ ]:
print("Hello")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import requests
import asyncio
import time
from dotenv import load_dotenv, find_dotenv
import logging

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

#import the KServer module from the src.k_data_helpers package
from pykaahma_linz import KServer

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
api_key = os.getenv('LINZ_API_KEY')

In [ ]:
rail_station_layer_id = "50318" #rail station 175 points
fences_layer_id = "50268" #NZ Fence Centrelines  
geodetic_marks_layer_id = "50787" # NZ Geodetic Marks 132,966 point features
nz_walking_biking_tracks_layer_id = "52100" #NZ Walking and Biking Tracks 29,187 polyline features
suburb_locality_table_id = "113761" #Suburb Locality - Population 3190 records

kserver = KServer.KServer(api_key)
print(kserver)
itm = kserver.content.get(fences_layer_id)
itm2 = kserver.content.get(nz_walking_biking_tracks_layer_id)
print(itm)
print(itm2)


In [ ]:
itm.title

In [ ]:
res = itm.query(count=2)
res.head()

In [ ]:
# Get a changeset from the geodetic marks layer
res = itm.get_changeset(from_time="2024-01-01T00:00:00Z")
logging.info(f"Total records returned {itm.title}: {res.shape[0]}")
res.head()


In [ ]:
waikato_polygon = {
        "coordinates": [
          [
            [
              174.30400216373914,
              -36.87399457472202
            ],
            [
              174.30400216373914,
              -38.83764306196984
            ],
            [
              176.83017911725346,
              -38.83764306196984
            ],
            [
              176.83017911725346,
              -36.87399457472202
            ],
            [
              174.30400216373914,
              -36.87399457472202
            ]
          ]
        ],
        "type": "Polygon"
      }

In [ ]:
output_folder = r"c:\temp\data"
job = itm.export("geodatabase", crs="EPSG:2193", extent=waikato_polygon,)
job.download(folder=output_folder)

In [ ]:
async def run_export(itm, export_format, crs, output_folder):
    job = itm.export(export_format, crs=crs)
    print(f"Started export job {job.id}")
    file_path = await job.download_async(output_folder)
    print(f"{job.id} downloaded to {file_path}")
    return file_path

async def export_multiple_items_async():
    output_folder = r"c:\temp\data\async"

    # Launch both jobs concurrently
    # print out the time taken to run both jobs
    logging.info("Starting multiple export jobs concurrently...")
    start_time = asyncio.get_event_loop().time()
    results = await asyncio.gather(
        run_export(itm, "geodatabase", "EPSG:2193", output_folder),
        run_export(itm2, "geodatabase", "EPSG:2193", output_folder),
    )
    end_time = asyncio.get_event_loop().time()
    logging.info(f"Both export jobs completed in {end_time - start_time:.2f} seconds")

    logging.info(f"Both exports completed: {str(results)}")

# In Jupyter or async context, use:
await export_multiple_items_async()

# In script:
# asyncio.run(export_multiple_jobs())




In [ ]:
def run_export_sync(itm, export_format, crs, output_folder):
    job = itm.export(export_format, crs=crs)
    print(f"Started export job {job.id}")
    file_path = job.download(output_folder)
    print(f"{job.id} downloaded to {file_path}")
    return file_path

def export_multiple_items_sync():
    output_folder = r"c:\temp\data\sync"

    logging.info("Starting multiple export jobs synchronously...")
    start_time = time.time()
    result1 = run_export_sync(itm, "geodatabase", "EPSG:2193", output_folder)
    result2 = run_export_sync(itm2, "geodatabase", "EPSG:2193", output_folder)
    end_time = time.time()
    logging.info(f"Both export jobs completed in {end_time - start_time:.2f} seconds")

    print(f"Both exports completed: {str([result1, result2])}")

# Call main_sync() in a normal script or notebook cell
export_multiple_items_sync()
